In [56]:
#Import Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from matplotlib import rcParams
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
#import seaborn as sns

from selenium import webdriver

print('Packages Imported')

Packages Imported


In [57]:
options = Options()

options.add_argument("--window-size=1920,1200")
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)
driver.get('https://brickset.com/sets/75192-1/Millennium-Falcon/')

In [73]:
#get name
def get_name(driver):
    name_element = driver.find_element(By.XPATH, "//dt[contains(text(), 'Name')]/following-sibling::dd")
    return name_element.text
print(get_name(driver))

Millennium Falcon


In [78]:
#get dollar price
def get_price_dollars(driver):
    price = 0
    price_element = driver.find_element(By.XPATH, "//dt[contains(text(), 'RRP')]/following-sibling::dd")
    for i in price_element.text.split('/'):
        if '$' in i:
            price = i.strip().replace('$','')
    return price

print(float(get_price_dollars(driver)))

849.99
